In [62]:
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer, AutoConfig, BatchEncoding, BertModel,LayoutLMModel
tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")
model = LayoutLMModel.from_pretrained("microsoft/layoutlm-base-uncased")
config = AutoConfig.from_pretrained("microsoft/layoutlm-base-uncased")
config

Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing LayoutLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


LayoutLMConfig {
  "_name_or_path": "microsoft/layoutlm-base-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_2d_position_embeddings": 1024,
  "max_position_embeddings": 512,
  "model_type": "layoutlm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [63]:
import json
from PIL import Image
# result = json.load(open('./data/processed/train/huy.jsonl'))
# result = json.load(open('data/processed/train/trinh.jsonl'))
# result = json.load(open('data/processed/train/kha.jsonl'))
result = json.load(open('data/processed/train/huong.jsonl'))
# result = json.load(open('data/processed/trinh.jsonl'))
# result = json.load(open('data/processed/trinh.jsonl'))
words = result['text']


In [64]:
w = result['img_sz']['width']
h = result['img_sz']['height']

In [105]:
bboxes = tuple([[ int(x[0][0]*1000/w),int(x[0][1]*1000/h) ,int(x[2][0]*1000/w),int(x[2][1]*1000/h)] for x in result['coord']])
# bboxes = tuple(bboxes)


# text label data_id coord
encoding

{'input_ids': tensor([[  101,  1060,  2050, 14684,  7570,  2072, 19710, 26478,  7570,  2050,
         12835, 12995, 15125,  1102, 10085,  2079,  5001, 10722,  7658,  2232,
          6887, 14194,  1011,  1011, 26478,  4907,  2064, 12731, 10085,  6390,
          2683,  5718,  8889,  2692, 12740,  2620,  6021,  2061,  7570,  2702,
          1024,  1011, 12436,  1045, 16215,  2072, 25283, 15876,  5063,  8254,
          2232,  2676,  1013,  5840,  1013,  2722,  1011, 12835,  4710,  2084,
          2290,  1010, 15125, 16371, 19710, 15125, 21025, 10448,  9543,  2232,
          1024, 22035,  2278, 14841,  2818,  1024, 10861,  6887, 14194,  9152,
         25311,  6887,  2080,  2084,  2232,  9152, 25311, 24110,  8026,  2232,
          1010,  9152, 25311,  8026,  2232,  2053,  2072, 19817,  2226,  1102,
         10441,  2072, 16215, 19098,  3070,  9152, 25311,  8026,  2232,  1010,
         16215, 19098,  3070,  1010,  9152, 25311,  6887, 14194,  6887,  2080,
          2084,  2232,  9152, 25311,  

In [104]:
def handel(word, coord, size):
    w,h = size
    normalized_word_boxes = tuple([[ int(x[0][0]*1000/w),int(x[0][1]*1000/h) ,int(x[2][0]*1000/w),int(x[2][1]*1000/h)] for x in coord])
    encoding = tokenizer(" ".join(words), return_tensors="pt")
    token_boxes = []
    tokens = []
    tokens = [tokenizer.cls_token] + tokens
    i =0
    maps = []
    maps.extend([[i]])

    for word, box in zip(words, normalized_word_boxes):
        word_tokens = tokenizer.tokenize(word)
        token_boxes.extend([box] * len(word_tokens))
        tokens.extend(word_tokens)
        i += 1
        maps.extend([[i]*len(word_tokens)])

    tokens += [tokenizer.sep_token]
    maps.extend([[i+1]])

    token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]
    token_type_ids = encoding["token_type_ids"]
    bbox = torch.tensor([token_boxes])
    return {'input_ids': input_ids, 'attention_mask': attention_mask,'token_type_ids':token_type_ids,'bbox':bbox , 'maps': maps}

handel(words,result['coord'],(w,h))

{'input_ids': tensor([[  101,  1060,  2050, 14684,  7570,  2072, 19710, 26478,  7570,  2050,
          12835, 12995, 15125,  1102, 10085,  2079,  5001, 10722,  7658,  2232,
           6887, 14194,  1011,  1011, 26478,  4907,  2064, 12731, 10085,  6390,
           2683,  5718,  8889,  2692, 12740,  2620,  6021,  2061,  7570,  2702,
           1024,  1011, 12436,  1045, 16215,  2072, 25283, 15876,  5063,  8254,
           2232,  2676,  1013,  5840,  1013,  2722,  1011, 12835,  4710,  2084,
           2290,  1010, 15125, 16371, 19710, 15125, 21025, 10448,  9543,  2232,
           1024, 22035,  2278, 14841,  2818,  1024, 10861,  6887, 14194,  9152,
          25311,  6887,  2080,  2084,  2232,  9152, 25311, 24110,  8026,  2232,
           1010,  9152, 25311,  8026,  2232,  2053,  2072, 19817,  2226,  1102,
          10441,  2072, 16215, 19098,  3070,  9152, 25311,  8026,  2232,  1010,
          16215, 19098,  3070,  1010,  9152, 25311,  6887, 14194,  6887,  2080,
           2084,  2232,  91

In [76]:
normalized_word_boxes = bboxes
# print(normalized_word_boxes.shape)
# words = ["Hello", "world"]
# normalized_word_boxes = [637, 773, 693, 782], [698, 773, 733, 782]
encoding = tokenizer(" ".join(words), return_tensors="pt")
token_boxes = []
tokens = []
tokens = [tokenizer.cls_token] + tokens
i =0
maps = []
maps.extend([[i]])

for word, box in zip(words, normalized_word_boxes):
    word_tokens = tokenizer.tokenize(word)
    token_boxes.extend([box] * len(word_tokens))
    tokens.extend(word_tokens)
    i += 1
    maps.extend([[i]*len(word_tokens)])

tokens += [tokenizer.sep_token]
maps.extend([[i+1]])
# add bounding boxes of cls + sep tokens

print(tokens)
token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]


# encoding = tokenizer(" ".join(words), return_tensors="pt")
input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]
token_type_ids = encoding["token_type_ids"]
bbox = torch.tensor([token_boxes])


print(input_ids.shape,attention_mask.shape,bbox.shape,token_type_ids.shape,np.array (maps).shape)
# print(pa['input_ids'].shape, pa['actual_bbox'].shape)
# sys.exit()
# token_labels = torch.tensor([1, 1, 0, 0]).unsqueeze(0)  # batch size of 1


outputs = model(
    input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids
)

# # last_hidden_states = outputs.last_hidden_state
# # loss = outputs.loss
# # logits = outputs.logits
last_hidden_state = outputs.last_hidden_state

# # loss = outputs.loss
# print(outputs.attentions)
print(last_hidden_state.shape)
# maps[1:-1]``

['[CLS]', 'x', '##a', 'chu', 'ho', '##i', 'viet', 'cong', 'ho', '##a', 'ng', '##hia', 'nam', 'đ', '##oc', 'do', 'lap', 'tu', 'han', '##h', 'ph', '##uc', '-', '-', 'cong', 'dan', 'can', 'cu', '##oc', '71', '##9', '07', '##00', '##0', '##40', '##8', '03', 'so', 'ho', 'ten', ':', '-', 'va', 'i', 'th', '##i', 'tran', 'hu', '##ong', 'sin', '##h', '27', '/', '04', '/', '1997', '-', 'ng', '##ay', 'than', '##g', ',', 'nam', 'nu', 'viet', 'nam', 'gi', '##oi', 'tin', '##h', ':', 'quo', '##c', 'ti', '##ch', ':', 'que', 'ph', '##uc', 'ni', '##nh', 'ph', '##o', 'than', '##h', 'ni', '##nh', 'quan', 'bin', '##h', ',', 'ni', '##nh', 'bin', '##h', 'no', '##i', 'tr', '##u', 'đ', '##oa', '##i', 'th', '##uo', '##ng', 'ni', '##nh', 'bin', '##h', ',', 'th', '##uo', '##ng', ',', 'ni', '##nh', 'ph', '##uc', 'ph', '##o', 'than', '##h', 'ni', '##nh', 'bin', '##h', 'co', 'gia', 'tri', 'đ', '##e', '27', '/', '04', '/', '202', '##2', '[SEP]']
torch.Size([1, 138]) torch.Size([1, 138]) torch.Size([1, 138, 4]) torch.

<ipython-input-76-10fb1457c4bc>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(input_ids.shape,attention_mask.shape,bbox.shape,token_type_ids.shape,np.array (maps).shape)


In [100]:
import torch.nn as nn
reduce_size = 256
# ln = nn.Linear(config.hidden_size, reduce_size)
# outputs = ln(last_hidden_state)
# last_hidden_state = outputs

# last_hidden_state.shape
# config.hidden_size


torch.Size([1, 138, 768])

In [77]:
i = 0
# reduce = torch.zeros(768)
reduce =[]
for g_token in  maps:
    ten = torch.zeros(reduce_size)
    for ele in g_token:
        # print(ele)
        ten += last_hidden_state[0][i]
        # i+=1
        # print(last_hidden_state[0][i])
        i+=1
    ten = ten/len(g_token)
    # print(ten)
    reduce.append(ten)
    # reduce = torch.cat((reduce,ten),-1)
# print(np.array(reduce).shape)
# print(reduce)
reduce=  torch.stack(reduce)
reduce.shape

RuntimeError: The size of tensor a (256) must match the size of tensor b (768) at non-singleton dimension 0

In [78]:
import torch.nn as nn
class RelationTagger(nn.Module):
    def __init__(self, n_fields, hidden_size, head_p_dropout=0.1):
        super().__init__()
        self.head = nn.Linear(hidden_size, hidden_size)
        self.tail = nn.Linear(hidden_size, hidden_size)
        self.field_embeddings = nn.Parameter(
            torch.rand(1, n_fields, hidden_size))
        self.W_label_0 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_label_1 = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, enc):

        enc_head = self.head(enc)
        enc_tail = self.tail(enc)

        batch_size = enc_tail.size(0)
        field_embeddings = self.field_embeddings.expand(batch_size, -1, -1)
        enc_head = torch.cat([field_embeddings, enc_head], dim=1)

        score_0 = torch.matmul(
            enc_head, self.W_label_0(enc_tail).transpose(1, 2))
        score_1 = torch.matmul(
            enc_head, self.W_label_1(enc_tail).transpose(1, 2))

        score = torch.cat(
            [
                score_0.unsqueeze(1),
                score_1.unsqueeze(1),
            ],
            dim=1,
        )
        return score


dropout = nn.Dropout(0.1)

In [87]:
loss = nn.CrossEntropyLoss()
rel_s = RelationTagger(
            hidden_size=config.hidden_size,
            n_fields=3,
        )
# rel_s = RelationTagger(
#             hidden_size=reduce_size,
#             n_fields=3,
#         )

rel_s = rel_s(dropout(last_hidden_state))
rel_s.shape
# S = rel_s
# S.shape
# last_hidden_state.shape

torch.Size([1, 2, 141, 138])

In [12]:

# S = torch.argmax(rel_s,dim =1)
s0,s1 = S[:,:,:3,:],S[:,:,3:,:]
s0.shape

torch.Size([1, 2, 3, 72])

In [122]:
# # torch.argmax(s0,dim =1).numpy()
# s1 =  s1[:,:,1:-1,1:-1]#reduce
# s0 = s0[:,:,:,1:-1]# reduce

from modeling.layoutlm import LayoutlmEmbeddings

layoutlmEmb = LayoutlmEmbeddings(config)

op = layoutlmEmb(input_ids=input_ids, bbox=bbox, token_type_ids = encoding["token_type_ids"])
# op.shape
# last_hidden_state
op

tensor([[[ 0.1383, -2.6283, -1.0686,  ...,  1.3360,  1.8693,  0.4746],
         [ 0.8942, -0.7362, -3.4043,  ..., -0.2805,  0.3122, -0.8545],
         [ 1.0675, -0.9226, -3.2150,  ...,  0.3079,  0.1756, -0.9706],
         ...,
         [ 1.0121, -0.0000, -0.0000,  ...,  0.7964, -0.1464, -1.0976],
         [ 0.4453, -1.2821, -1.0664,  ...,  0.6701,  0.6806, -2.4718],
         [-0.4023, -1.8295,  0.0217,  ...,  0.3073,  2.0244, -2.3951]]],
       grad_fn=<MulBackward0>)

In [14]:
rel_g = RelationTagger(
            hidden_size=reduce_size,
            n_fields=3,
        )

rel_g = rel_g(dropout(reduce.unsqueeze(0)))
# rel_s.shape
G = rel_g
# S = torch.argmax(rel_s,dim =1)
g0,g1 = G[:,:,:3,:],G[:,:,3:,:]
# torch.argmax(s0,dim =1).numpy()
# g1 =  g1[:,:,1:-1,1:-1]#reduce
# g0 = g0[:,:,:,1:-1]# reduce
g1.shape
# torch.argmax(s0,dim =1)

torch.Size([1, 2, 72, 72])

In [15]:
graph = np.array(result['label'])
graph[0, :3, :]


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0]])

In [16]:
label = [ [0] + list(x) + [0] for x in list(graph[0, :3, :])]
np.array(label).shape
#done
# label.shape

(3, 72)

In [17]:
def extend_matrix(matrix):

    matrix_s = [ [0] + list(x) + [0] for x in list(matrix)] 
    t_m =  list( np.zeros_like(matrix_s[0]))
    _s = [t_m] + list(matrix_s) + [t_m]

    return np.array(_s)
# np.array(_s)
def extend_label(label_):
    label = [ [0] + list(x) + [0] for x in list(label_)]
    return np.array (label)
# label =  
label = torch.tensor(extend_label(graph[0, :3, :])).unsqueeze(0)
matrix_s = torch.tensor(extend_matrix(graph[0, 3:, :])).unsqueeze(0)
matrix_g = torch.tensor(extend_matrix(graph[1, 3:, :])).unsqueeze(0)
# np.array(extend_matrix(graph[0, 3:, :])).shape

loss_label_s = loss(s0, label)
loss_matrix_s = loss(s1,matrix_s)
loss_matrix_g = loss(g1,matrix_g)
print(loss_label_s, loss_matrix_s, loss_matrix_g)
loss_label_s + loss_matrix_s + loss_matrix_g

tensor(0.3553, grad_fn=<NllLoss2DBackward0>) tensor(0.7797, grad_fn=<NllLoss2DBackward0>) tensor(0.8225, grad_fn=<NllLoss2DBackward0>)


tensor(1.9575, grad_fn=<AddBackward0>)

Ground Truth

In [18]:
from graph_stuff import get_strings, get_qa
import networkx as nx
from jsonmerge import merge
text_tok =  [tokenizer.cls_token] + result['text'] + [tokenizer.sep_token]
label = label.squeeze(0)
S_ = extend_matrix(graph[0, 3:, :])
G_ = extend_matrix(graph[1, 3:, :])
question_heads = [i for i, ele in enumerate(label[0]) if ele != 0]
answer_heads = [i for i, ele in enumerate(label[1]) if ele != 0]
header_heads = [i for i, ele in enumerate(label[2]) if ele != 0]

# G = nx.Graph(S_)
ques = get_strings(question_heads, text_tok, S_)
print(ques)
ans = get_strings(answer_heads, text_tok, S_)
print(ans)
resul_ = {}
for ques_idx in question_heads:
    q_, a_ = get_qa(ques_idx, ques, ans, G_)
    resul_ = merge(resul_, {q_: a_})
    # print(get_ques_ans(ques_idx, ques,ans))
print(resul_)

[[24, 'Số'], [25, 'Ho và tên:'], [35, 'Ngày tháng, năm'], [41, 'Giới tính:'], [43, 'Quốc tịch:'], [45, 'Quê quán'], [55, 'Nơi thường'], [67, 'Có giá tri']]
[[23, '03 719 07000408'], [31, 'TRẦN THỊ HƯƠNG'], [33, 'sinh 27/04/1997'], [38, 'Nữ'], [39, 'Việt Nam'], [47, 'Ninh Phúc Thành phố Ninh Bình, Ninh Binh'], [56, 'trú Đoài Thương, Ninh Phúc Thành phố Ninh Bình, Ninh Binh'], [70, 'đế 27/04/2022']]
{'Số': '03 719 07000408', 'Ho và tên:': 'TRẦN THỊ HƯƠNG', 'Ngày tháng, năm': 'sinh 27/04/1997', 'Giới tính:': 'Nữ', 'Quốc tịch:': 'Việt Nam', 'Quê quán': 'Ninh Phúc Thành phố Ninh Bình, Ninh Binh', 'Nơi thường': 'trú Đoài Thương, Ninh Phúc Thành phố Ninh Bình, Ninh Binh', 'Có giá tri': 'đế 27/04/2022'}


In [133]:
heads =  question_heads
data = text_tok
graph =  S_

list_bbox =[]
scale_bboxes = tuple([[ x[0][0]/w,x[0][1]/h ,x[2][0]/w,x[2][1]/h] for x in result['coord']])
ex_bboxes =[[0, 0, 0, 0]] + list(scale_bboxes) + [[1, 1, 1, 1]]


In [165]:
# G = nx.Graph(graph_s[0,3:,:]) # s




* Predict

In [142]:
pred_label = torch.argmax(s0,dim=1).squeeze(0)
pred_matrix_s = torch.argmax(s1,dim=1).squeeze(0)
pred_matrix_g = torch.argmax(g1,dim=1).squeeze(0)
pred_label.shape
# pred_matrix_s
pred_question_heads = [i for i, ele in enumerate(pred_label[0]) if ele != 0]
pred_answer_heads = [i for i, ele in enumerate(pred_label[1]) if ele != 0]
pred_header_heads = [i for i, ele in enumerate(pred_label[2]) if ele != 0]
pred_S = np.array([list(x) for x in np.array(pred_matrix_s.numpy())])
pred_G = np.array([list(x) for x in np.array(pred_matrix_g.numpy())])


In [167]:
# abs(sum(temp)/len(temp) - sum(temp_)/len(temp_))

# b_loss.backward()
def b_loss(graph,ex_bboxes,list_heads):
    question_heads, answer_heads,pred_answer_heads ,pred_answer_heads  = list_heads
    def bbox_loss(graph,ex_bboxes,heads):
        temp = []
        G = nx.Graph(graph) # s
        for index in heads:
            dfs = list(nx.dfs_edges(G, source=int(index)))
            dfs
            if  dfs == []:
                header = [int(index)]
            else: header =  [dfs[0][0]] + [x[1]  for i,x in enumerate (dfs)]
            str_ = ''
            list_temp = []
            for i in header:
                str_ += ' ' + data[int(i)] 
                [x1,y1,x2,y2] = ex_bboxes[int(i)]
                # print(abs(x2 -x1) + abs(y2 -y1) )
                list_temp.append(abs(x2 -x1) + abs(y2 -y1))
                assert i <= len(data)
            temp.append(sum(list_temp)/len(list_temp))
        try: 
            return sum(temp)/len(temp)
        except ZeroDivisionError:
            return 0
    
    que_loss = bbox_loss(S_, ex_bboxes,question_heads)
    ans_loss = bbox_loss(S_, ex_bboxes,answer_heads)
    pred_ques_loss =  bbox_loss(pred_S, ex_bboxes,pred_question_heads)
    pred_ans_loss =  bbox_loss(pred_S, ex_bboxes,pred_answer_heads)
    return abs(pred_ques_loss - que_loss) +abs(pred_ans_loss - ans_loss)

In [95]:

# nn.BCEWithLogitsLoss()

# ques = get_strings(pred_question_heads, text_tok, pred_matrix_s)
# print(ques)
# ans = get_strings(pred_answer_heads, text_tok, pred_matrix_s)
# print(ans)
# pred_S =  np.array(pred_matrix_s.numpy())
# # list(pred_S)

# G = nx.Graph(pred_S)

ques = get_strings(pred_question_heads, text_tok, pred_S)
# print(np.shape(ques))

ans = get_strings(answer_heads, text_tok, S_)
# print(np.shape(ans))

for ques_idx in pred_question_heads:
    G = nx.Graph(pred_G) # group
    dfs = list(nx.dfs_edges(G, source=int(ques_idx)))
    # print(dfs)
    q,a = dfs[0]
    qu_s =  [qs[1] for qs in ques if q in qs ]
    an_s=  [as_[1] for as_ in ans if a in as_ ]
    if len(qu_s)== len(an_s):
        print(qu_s[0], an_s[0])
    else: print( f'ques: {qu_s} \n ans: {an_s}' )

ques: ['[CLS] XÃ CHỦ HỘI VIỆT CỘNG HÒA NGHIA NAM Độc do lập Tự Hạnh CÔNG phúc DÂN - CĂN CƯỚC 07000408 - tên: 03 THỊ Số 719 TRẦN Ho - HƯƠNG I Ngày và tháng, sinh 27/04/1997 năm - Giới Nữ Việt Nam tính: Quốc tịch: phố Quê Phúc Thành Ninh Ninh quán Nơi Bình, Ninh Binh trú Đoài thường Thương, Ninh Phúc Bình, Ninh phố Thành Ninh Binh Có đế 27/04/2022 giá [SEP] tri'] 
 ans: []
ques: ['[SEP] [CLS] XÃ CHỦ HỘI VIỆT CỘNG HÒA NGHIA NAM Độc do lập Tự Hạnh CÔNG phúc DÂN - CĂN CƯỚC 07000408 - tên: 03 THỊ Số 719 TRẦN Ho - HƯƠNG I Ngày và tháng, sinh 27/04/1997 năm - Giới Nữ Việt Nam tính: Quốc tịch: phố Quê Phúc Thành Ninh Ninh quán Nơi Bình, Ninh Binh trú Đoài thường Thương, Ninh Phúc Bình, Ninh phố Thành Ninh Binh Có đế 27/04/2022 giá tri'] 
 ans: []
